<a href="https://colab.research.google.com/github/sohaum/Text-Summarizer/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

In [2]:
from datasets import load_dataset

In [3]:
multinews=load_dataset("multi_news",split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [4]:
multinews.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer=AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
multi_news=multinews.train_test_split(test_size=0.2)

In [8]:
prefix="summarize:"
def process_function(examples):
  inputs=[prefix+doc for doc in examples["document"]]
  model_input=tokenizer(inputs,max_length=1024,truncation=True)
  labels=tokenizer(text=examples["summary"],max_length=128,truncation=True)
  model_input["labels"]=labels["input_ids"]
  return model_input

In [9]:
tokenized_multi_news=multi_news.map(process_function,batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM,  Seq2SeqTrainingArguments, Seq2SeqTrainer
DataCollator=DataCollatorForSeq2Seq(tokenizer,model=AutoModelForSeq2SeqLM.from_pretrained("t5-small"))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
training_args=Seq2SeqTrainingArguments(
    output_dir="summarizer",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [12]:
trainer=Seq2SeqTrainer(
    model=AutoModelForSeq2SeqLM.from_pretrained("t5-small"),
    args=training_args,
    train_dataset=tokenized_multi_news["train"],
    eval_dataset=tokenized_multi_news["test"],
    data_collator=DataCollator,
    tokenizer=tokenizer,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.880195
2,3.234800,2.803689
3,3.234800,2.771996
4,2.993400,2.752393
5,2.993400,2.740294
6,2.949400,2.732267
7,2.949400,2.727070
8,2.918200,2.722925
9,2.907700,2.720424
10,2.907700,2.720047


TrainOutput(global_step=2820, training_loss=2.9899443065021054, metrics={'train_runtime': 2681.4089, 'train_samples_per_second': 16.771, 'train_steps_per_second': 1.052, 'total_flos': 1.217221471617024e+16, 'train_loss': 2.9899443065021054, 'epoch': 10.0})

In [14]:
text="""transformer is a neural network architecture used for performing machine learning tasks. In 2017 Vaswani et al. published a paper ” Attention is All You Need” in which the transformers architecture was introduced. Since then, transformers have been widely adopted and extended for various machine learning tasks beyond NLP. The article explores the architecture, working and applications of transformer. Transformer Architecture is a model that uses self-attention that transforms one whole sentence into a single sentence. This is a big shift from how older models work step by step, and it helps overcome the challenges seen in models like RNNs and LSTMs. Need For Transformer The transformer neural network was a revolution in the field of machine learning. Let’s have look at the reasons why transformer architecture is needed: RNN suffers from the vanishing gradient problem which causes long-term memory loss. RNN does the processing of text sequentially, which means if there is a long sentence like- ‘XYZ went to France in 2019 when there were no cases of COVID and there he met the president of that country.‘ Now, if we ask that here ‘that country’ refers to which place? RNN will not be able to recall that the country was ‘France’ because it has encountered the word ‘France’ long before. The sequential nature of processing means that the model was trained at the word level, not at the sentence level. The gradients carry information used in the RNN parameter update and when the gradient becomes smaller, then no real learning is done. With the addition of a few more memory cells and resolving the vanishing gradients issue, the problem regarding long-term memory loss was resolved to some extent. However, the problem with sequential processing remained because RNN was unable to process the intact sentence at once. Rather than processing in parallel, it processed words one by one. This issue can’t be addressed in LSTMs due to their sequential design. In LSTMs, we use the static embedding method, which suggests that without knowing the context of a word we embed it to some n-dimensional vector. But if the context changes, the meaning also changes. Ex: There is a word -’ Point’, and we use it in two different contexts given below The needle has a sharp point. It is not polite to point at people. Here, the word ‘Point’ has two different contexts in both of the sentences, but when embedding is done the context is not taken into consideration. Therefore, there was a need for a different architecture — Transformer. Architecture and Working of Transformers Positional Encoding Transformers lack an inherent understanding of the sequential order of elements, positional encodings are incorporated into the input embeddings. These encodings serve to convey information about the specific positions of tokens within the sequence. Position-wise Feedforward Networks The feedforward network consists if a fully connected layer for followed by non-linear activation function, such as Rectified Linear Unit (ReLU). The model captures and process features at different position in the sequence. These networks in both the encoder and decoder operate independently on each position. Attention Mechanism The attention mechanism in transformers employs a scaled dot-product approach, where the computation involves scaled dot products between the query, key, and value vectors. This produces weighted values that are then summed to yield the attention output. To enhance the model’s capacity to capture diverse relationships within the input, the multi-head attention mechanism is introduced. This involves applying the attention mechanism multiple times concurrently, each with distinct learned linear projections of the input. The resulting outputs from these parallel computations are concatenated and undergo a linear transformation to generate the final attention result. This multi-head approach allows the model to attend to different aspects of the input sequence simultaneously, promoting richer and more nuanced representations. Encoder-Decoder Architecture Encoder-Decoder architecture is the main component of transformer architecture. The encoder block turns the sequence of input words into a vector and a Decoder converts a vector into a sequence. Eg: A text in French processed into its English equivalent can be: Je suis étudiant –> I am a student.
Transformer Architecture The encoder architecture has two layers: Self Attention and Feed Forward.
The encoder’s inputs first pass by a self-attention layer and then the outputs of the self-attention layer are fed to a feed-forward neural network.
Sequential data has temporal characteristics. It signifies that each word holds some position concerning the other. For example, let’s take a sentence- ‘The cat didn’t chase the mouse, because it was not hungry‘. Here, we can easily tell that ‘it’ is referring to the cat, but it is not as simple for an algorithm. When the model is processing the word ‘it’, self-attention allows it to associate ‘it’ with ‘cat’. Self-attention is the method to reformulate the representation based on all other words of the sentence. Encoder and Decoder Layer Architecture of Transformer The decoder architecture has three layers: Self Attention, Encoder-decoder attention, and Feed Forward. The decoder has both the self-attention and feed-forward layer, which is also present in the encoder, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence. There are six layers of encoders and decoders in Transformer architecture. In the bottom encoder, word embeddings are performed, and each word is transformed into a vector of size 512. The input to the other encoders would be the output of the encoder that’s directly below. The various layers of the encoder are for discovering the NLP pipeline. Like- the first layer is used for Part of speech tags, the second layer for constituents, the third layer for dependencies, the fourth layer for semantic roles, the fifth for coreference, and the sixth for relations. Applications of Transformer Some of the applications of transformers are: Transformers is used for NLP tasks like, machine translation, text summarization, name entity recognition and sentimental analysis. Another application is speech recognition system, where audio signals are processed to provide transcribed text. The application transformers in only limited to NLP, it is also used for computer vision tasks such as image classification, object detection, image generation. Transformers are also used in recommendation system to provide customized recommendations.
Transformers can used for text and music generation also."""

In [15]:
input_ids=tokenizer(text,max_length=1024,truncation=True,return_tensors='pt')["input_ids"]
input_ids=input_ids.to('cuda')

In [23]:
import torch
with torch.no_grad():
  if trainer.model.device.type=='cuda': # Use the model from the trainer object
    output=trainer.model.generate(input_ids,max_length=147,num_beams=5)

summary_ids=output[0].tolist()
summary=tokenizer.decode(summary_ids,skip_special_tokens=True)
print(summary)

– Transformer Architecture is a neural network architecture used for performing machine learning tasks. In 2017, Vaswani et al. published a paper titled Attention is All You Need. Transformer Architecture is a model that uses self-attention that transforms one whole sentence into a single sentence, which means if there is a long sentence like- ‘XYZ went to France in 2019 when there were no cases of COVID and there he met the president of that country.’ RNN suffers from the vanishing gradient


In [25]:
summary

'– Transformer Architecture is a neural network architecture used for performing machine learning tasks. In 2017, Vaswani et al. published a paper titled Attention is All You Need. Transformer Architecture is a model that uses self-attention that transforms one whole sentence into a single sentence, which means if there is a long sentence like- ‘XYZ went to France in 2019 when there were no cases of COVID and there he met the president of that country.’ RNN suffers from the vanishing gradient'

In [18]:
ref_summary="""Transformer is a neural network architecture used for machine learning tasks, introduced in 2017 by Vaswani et al. It uses self-attention to transform a whole sentence into a single sentence, overcoming challenges in models like RNNs and LSTMs. Transformers overcome the vanishing gradient problem, which causes long-term memory loss in RNNs. RNNs process text sequentially, causing problems with word-level processing and vanishing gradients.
Transformers address this issue by incorporating positional encodings into input embeddings, which convey information about the specific positions of tokens within the sequence.
The feedforward network consists of a fully connected layer and a non-linear activation function, such as Rectified Linear Unit (ReLU), which capture and process features at different positions in the sequence. Transformers also employ a scaled dot-product approach for attention, which is enhanced by the multi-head attention mechanism, which applies the attention mechanism multiple times concurrently, each with distinct learned linear projections of the input."""

In [21]:
!pip install rouge

In [24]:
from rouge import Rouge
rouge=Rouge()
scores=rouge.get_scores(summary,ref_summary)
scores

[{'rouge-1': {'r': 0.25225225225225223,
   'p': 0.417910447761194,
   'f': 0.31460673687855073},
  'rouge-2': {'r': 0.11409395973154363,
   'p': 0.21794871794871795,
   'f': 0.14977973117196156},
  'rouge-l': {'r': 0.24324324324324326,
   'p': 0.40298507462686567,
   'f': 0.3033707818223709}}]

In [26]:
trainer.save_model()

In [28]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
model.save_pretrained("final")